In [18]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
import ast
import numpy as np
import os
import ast
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os.path
from datetime import datetime
from collections import Counter

nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')

os.chdir("/home/domantas/Desktop/University/bachelor_url_categorization")
month = "december"
char_blacklist = list(chr(i) for i in range(32, 127) if i <= 64 or i >= 91 and i <= 96 or i >= 123)
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(char_blacklist)
english_vocab = set(w.lower() for w in nltk.corpus.words.words())
english_tolerance = 50
english_confidence = []
words_threshold = 10
top = 2500
toker = RegexpTokenizer(r'((?<=[^\w\s])\w(?=[^\w\s])|(\W))+', gaps=True)
words_frequency = {}

file = 'Datasets/URL-categorization-DFE.csv'
df = pd.read_csv(file)[['main_category', 'main_category:confidence', 'url']][:50]
df = df[(df['main_category'] != 'Not_working') & (df['main_category:confidence'] > 0.5)]
df['tokenized_words'] = ''
counter = 0
for i, row in df.iterrows():
    counter += 1
    print("{}, {}/{}; Time: {}".format(row['url'], counter, len(df), str(datetime.now())))
    try:
        hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
           'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
           'Accept-Encoding': 'none',
           'Accept-Language': 'en-US,en;q=0.8',
           'Connection': 'keep-alive'}
        req = urllib.request.Request('http://' + row['url'], headers=hdr)
        html = urlopen(req, timeout=15).read()
#             html = urlopen('http://' + row['url'], timeout=15).read()
    except:
        print("{} Failed".format(row['url']))
        continue

    soup = BeautifulSoup(html, "html.parser")
    [tag.decompose() for tag in soup("script")]
    [tag.decompose() for tag in soup("style")]
    text = soup.get_text()
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk.lower() for chunk in chunks if chunk)
    # Tokenize text
    tokens = [token.lower() for token in toker.tokenize(text)]
    # Remove stopwords
    wnl = WordNetLemmatizer()
    tokens = [token.lower() for token in toker.tokenize(text)]
    tokens_stopwords = [w.lower() for w in tokens if w not in stopwords and len(w) >= 3 and w[0] not in char_blacklist]
    tokens_lemmatize = [wnl.lemmatize(token) for token in tokens_stopwords]
    # Calculate percentage of english words
    english_tokens = []
    for word in tokens_lemmatize:
        english_tokens.append(word.lower()) if word.lower() in english_vocab else ''
    english_confidence = len(english_tokens) / len(tokens_lemmatize) * 100 if len(english_tokens) > 0 else 0
    if len(english_tokens) < words_threshold or english_confidence < english_tolerance:
        continue
    # Track frequent words list for each category
    if row['main_category'] not in [c for c in words_frequency.keys()]:
        words_frequency[row['main_category']] = []
    words_frequency[row['main_category']] = words_frequency[row['main_category']] + english_tokens

    df.at[i, 'tokenized_words'] = english_tokens if english_confidence > english_tolerance else ''
    df.at[i, 'english:confidence'] = english_confidence

df = df[df['tokenized_words'] != '']
df.to_csv("Datasets/full_data_{}_2.csv".format(month))

# Count words frequency for each category
for cat in words_frequency.keys():
    frequency = nltk.FreqDist(w for w in words_frequency[cat]).most_common(top)
    words_frequency[cat] = [word for word, number in frequency]

# Remove chunk words
from math import floor
words = []
for category in words_frequency.keys():
    words.extend(words_frequency[category][0:20])
words_counter = Counter(words)
chunk_words = [x for x in words_counter if words_counter[x] >= 7]
words_filter = {x : words_counter[x] for x in words_counter if words_counter[x] >= 7}
for cat in words_frequency.keys():
    words_frequency[cat] = [word for word in words_frequency[cat] if word not in chunk_words]
# Create labels and features set for ML
features = np.zeros(df.shape[0] * top).reshape(df.shape[0], top)
labels = np.zeros(df.shape[0])
counter = 0
for i, row in df.iterrows():
    c = [word for word, word_count in Counter(row['tokenized_words']).most_common(top)]
    labels[counter] = list(set(df['main_category'].values)).index(row['main_category'])
    for word in c:
        if word in words_frequency[row['main_category']]:
            features[counter][words_frequency[row['main_category']].index(word)] = 1
    counter += 1
# Features and labels splitting to training and testing data
from sklearn.metrics import accuracy_score
from scipy.sparse import coo_matrix
X_sparse = coo_matrix(features)

from sklearn.utils import shuffle
X, X_sparse, y = shuffle(features, X_sparse, labels, random_state=0)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Train and validate data using ML algorithms
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
predictions = lr.predict(X_test)
score = lr.score(X_test, y_test)
print('LogisticRegression')
print('Score: ', score)
print('Top: ', top)
print('Tolerance: ', english_tolerance)
print('Dataset length: ', df.shape[0])
print()
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
predictions = dtc.predict(X_test)
score = dtc.score(X_test, y_test)
print('DecisionTreeClassifier')
print('Score: ', score)
print('Top: ', top)
print('Tolerance: ', english_tolerance)
print('Dataset length: ', df.shape[0])
print()
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
score = clf.score(X_test, y_test)
print('SVM')
print('Score: ', score)
print('Top: ', top)
print('Tolerance: ', english_tolerance)
print('Dataset length: ', df.shape[0])

#     # Save models
#     from sklearn.externals import joblib
#     filename = "Models/LR_model_{}_2.joblib".format(month)
#     if not os.path.isfile(filename):
#         joblib.dump(lr, filename)

#     import pickle
#     words_filename = "Models/word_frequency_{}_2.picle".format(month)
#     if not os.path.isfile(words_filename):
#         pickle_out = open(words_filename,"wb")
#         pickle.dump(words_frequency, pickle_out)
#         pickle_out.close()

#     filename = "Models/LR_maxtrain_{}_2.joblib".format(month)
#     if not os.path.isfile(filename):
#         from sklearn.linear_model import LogisticRegression
#         lr = LogisticRegression()
#         lr.fit(X, y)
#         joblib.dump(lr, filename)



[nltk_data] Downloading package stopwords to
[nltk_data]     /home/domantas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/domantas/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /home/domantas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


000webhost.com, 1/26; Time: 2018-12-31 01:36:06.024147
022menchuang.net, 2/26; Time: 2018-12-31 01:36:07.375099
022menchuang.net Failed
055firenze.it, 3/26; Time: 2018-12-31 01:36:08.414930
0calc.com, 4/26; Time: 2018-12-31 01:36:09.403505
1-x-bet.com, 5/26; Time: 2018-12-31 01:36:09.882065
1000-instrumentov.ru, 6/26; Time: 2018-12-31 01:36:10.533373


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


1000.menu, 7/26; Time: 2018-12-31 01:36:11.301089
1000ps.at, 8/26; Time: 2018-12-31 01:36:11.760325
10086.cn, 9/26; Time: 2018-12-31 01:36:13.201771
100diet.net, 10/26; Time: 2018-12-31 01:36:14.500352
104.com, 11/26; Time: 2018-12-31 01:36:14.997503
1091.co.jp, 12/26; Time: 2018-12-31 01:36:16.439481
10bet.com, 13/26; Time: 2018-12-31 01:36:18.738322
10betlive.com, 14/26; Time: 2018-12-31 01:36:20.985580
10betlive.com Failed
1100ad.com, 15/26; Time: 2018-12-31 01:36:22.712337
111.com.cn, 16/26; Time: 2018-12-31 01:36:23.348370
1111.com.tw, 17/26; Time: 2018-12-31 01:36:29.751039
111cn.net, 18/26; Time: 2018-12-31 01:36:37.869280
112-magazin.de, 19/26; Time: 2018-12-31 01:36:46.463659
121ware.com, 20/26; Time: 2018-12-31 01:36:52.331413
123project.ir, 21/26; Time: 2018-12-31 01:36:55.883538
123vidz.com, 22/26; Time: 2018-12-31 01:36:58.645960
1337x.to, 23/26; Time: 2018-12-31 01:36:59.996471
13980.com, 24/26; Time: 2018-12-31 01:37:00.245594
13980.com Failed
140online.com, 25/26; Time:

/home/domantas/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/domantas/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [24]:
words_frequency

{'Internet_and_Telecom': ['resource',
  'control',
  'panel',
  'file',
  'learn',
  'disk',
  'full',
  'popular',
  'live',
  'offer',
  'earn',
  'guarantee',
  'option',
  'top',
  'world',
  'access',
  'need',
  'click',
  'installer',
  'make',
  'easy',
  'site',
  'content',
  'limited',
  'day',
  'united',
  'state',
  'suomi',
  'affiliate',
  'money',
  'company',
  'ready',
  'one',
  'contact',
  'form',
  'best',
  'course',
  'build',
  'available',
  'may',
  'view',
  'review',
  'order',
  'cost',
  'quality',
  'completely',
  'variety',
  'without',
  'proof',
  'every',
  'latest',
  'upgrade',
  'extra',
  'script',
  'gallery',
  'step',
  'template',
  'knowledge',
  'choose',
  'something',
  'amazing',
  'single',
  'instant',
  'activation',
  'register',
  'simple',
  'within',
  'provide',
  'place',
  'possible',
  'platform',
  'also',
  'starting',
  'become',
  'tutorial',
  'main',
  'come',
  'business',
  'level',
  'backup',
  'home',
  'page',
  

In [17]:
features = np.zeros(df.shape[0] * top).reshape(df.shape[0], top)
labels = np.zeros(df.shape[0])
counter = 0
for i, row in df.iterrows():
    c = [word for word, word_count in Counter(row['tokenized_words']).most_common(top)]
    labels[counter] = list(set(df['main_category'].values)).index(row['main_category'])
    for word in c:
        if word in words_frequency[row['main_category']]:
            features[counter][words_frequency[row['main_category']].index(word)] = 1
    counter += 1

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


In [ ]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
import ast
import numpy as np
import os
import ast
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os.path
from datetime import datetime
from collections import Counter

nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')

In [ ]:
os.chdir("/home/domantas/Desktop/University/bachelor_url_categorization")
month = "december"
char_blacklist = list(chr(i) for i in range(32, 127) if i <= 64 or i >= 91 and i <= 96 or i >= 123)
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(char_blacklist)
english_vocab = set(w.lower() for w in nltk.corpus.words.words())
english_tolerance = 50
english_confidence = []
words_threshold = 15
top = 2500
toker = RegexpTokenizer(r'((?<=[^\w\s])\w(?=[^\w\s])|(\W))+', gaps=True)
words_frequency = {}

In [ ]:
def no_filter_data():
    file = 'Datasets/URL-categorization-DFE.csv'
    df = pd.read_csv(file)[['main_category', 'main_category:confidence', 'url']]
    df = df[(df['main_category'] != 'Not_working') & (df['main_category:confidence'] > 0.5)][:200]
    df['tokenized_words'] = ''
    counter = 0
    for i, row in df.iterrows():
        counter += 1
        print("{}, {}/{}; Time: {}".format(row['url'], counter, len(df), str(datetime.now())))
        try:
            hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
               'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
               'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
               'Accept-Encoding': 'none',
               'Accept-Language': 'en-US,en;q=0.8',
               'Connection': 'keep-alive'}
            req = urllib.request.Request('http://' + row['url'], headers=hdr)
            html = urlopen(req, timeout=15).read()
        except:
            print("{} Failed".format(row['url']))
            continue

        soup = BeautifulSoup(html, "html.parser")
        [tag.decompose() for tag in soup("script")]
        [tag.decompose() for tag in soup("style")]
        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk.lower() for chunk in chunks if chunk)
        # Tokenize text
        tokens = [token.lower() for token in toker.tokenize(text)]
        # Remove stopwords
        wnl = WordNetLemmatizer()
        tokens_stopwords = [w.lower() for w in tokens if w not in stopwords and len(w) >= 3 and w[0] not in char_blacklist]
        tokens_lemmatize = [wnl.lemmatize(token) for token in tokens_stopwords]
        # Calculate percentage of english words
        english_tokens = []
        for word in tokens_lemmatize:
            if word.lower() in english_vocab:
                english_tokens.append(word.lower())
        english_confidence = len(english_tokens) / len(tokens_lemmatize) * 100 if len(english_tokens) > 0 else 0
        if len(english_tokens) < words_threshold or english_confidence < english_tolerance:
            continue
        # Track frequent words list for each category
        if row['main_category'] not in [c for c in words_frequency.keys()]:
            words_frequency[cat] = []
        words_frequency[row['main_category']] = words_frequency[row['main_category']] + english_tokens
        
        df.at[i, 'tokenized_words'] = english_tokens
        df.at[i, 'english:confidence'] = english_confidence
        
    df = df[~df['tokenized_words'].isnull()]
    df.to_csv("Datasets/full_data_{}.csv".format(month))
if not os.path.isfile("Datasets/full_data_{}.csv".format(month)):
    no_filter_data()
df = pd.read_csv("Datasets/full_data_{}.csv".format(month))
df = df[df['english:confidence'] > english_tolerance]

In [ ]:
import pickle
pickle_in = open("Models/December/word_frequency_December.picle","rb")
words_frequency = pickle.load(pickle_in)

# Count words frequency for each category

In [ ]:
wnl = WordNetLemmatizer()
for cat in words_frequency.keys():
    frequency = nltk.FreqDist(w.lower() for w in words_frequency[cat]).most_common(top)
    words_frequency[cat] = [wnl.lemmatize(word) for word, number in frequency]

In [ ]:
words_frequency

# Remove common words in all categories

In [ ]:
from math import floor
words = []
for category in words_frequency.keys():
    words.extend(words_frequency[category][0:20])
words_counter = Counter(words)
chunk_words = [x for x in words_counter if words_counter[x] >= 7]
words_filter = {x : words_counter[x] for x in words_counter if words_counter[x] >= 7}
for cat in words_frequency.keys():
    words_frequency[cat] = [word for word in words_frequency[cat] if word not in chunk_words]

# Create labels and features set for ML

In [ ]:
features = np.zeros(df.shape[0] * top).reshape(df.shape[0], top)
labels = np.zeros(df.shape[0])
counter = 0
for i, row in df.iterrows():
    print(counter) if row['url'] == '10bet.com' else ''
    c = [word for word, word_count in Counter(ast.literal_eval(row['tokenized_words'])).most_common(top)]
    labels[counter] = list(set(df['main_category'].values)).index(row['main_category'])
    for word in c:
        if word in words_frequency[row['main_category']]:
            features[counter][words_frequency[row['main_category']].index(word)] = 1
    counter += 1

from sklearn.metrics import accuracy_score
from scipy.sparse import coo_matrix
X_sparse = coo_matrix(features)

from sklearn.utils import shuffle
X, X_sparse, y = shuffle(features, X_sparse, labels, random_state=0)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
predictions = lr.predict(X_test)

In [ ]:
labels[3]

In [ ]:
predictions[3]

In [ ]:
list(set(df['main_category'].values))[14]

# Train and validate data using ML algorithms

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
predictions = lr.predict(X_test)
score = lr.score(X_test, y_test)
print('LogisticRegression')
print('Score: ', score)
print('Top: ', top)
print('Tolerance: ', english_tolerance)
print('Dataset length: ', df.shape[0])
print()
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
predictions = dtc.predict(X_test)
score = dtc.score(X_test, y_test)
print('DecisionTreeClassifier')
print('Score: ', score)
print('Top: ', top)
print('Tolerance: ', english_tolerance)
print('Dataset length: ', df.shape[0])
print()
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
score = clf.score(X_test, y_test)
print('SVM')
print('Score: ', score)
print('Top: ', top)
print('Tolerance: ', english_tolerance)
print('Dataset length: ', df.shape[0])

# Save models

In [ ]:
month = "December"
from sklearn.externals import joblib

filename = "Models/{}/LR_model_{}.joblib".format(month, month)
if not os.path.isfile(filename):
    joblib.dump(lr, filename)

filename = "Models/{}/LR_maxtrain_{}.joblib".format(month, month)
if not os.path.isfile(filename):
    from sklearn.linear_model import LogisticRegression
    lr = LogisticRegression()
    lr.fit(X, y)
    joblib.dump(lr, filename)

filename = "Models/{}/SVM_model_{}.joblib".format(month, month)
if not os.path.isfile(filename):
    joblib.dump(clf, filename)
    
import pickle
words_filename = "Models/{}/word_frequency_{}.picle".format(month, month)
if not os.path.isfile(words_filename):
    pickle_out = open(words_filename,"wb")
    pickle.dump(words_frequency, pickle_out)
    pickle_out.close()